<a href="https://colab.research.google.com/github/kalyyanpavan1/EVA3-Project-Folders/blob/Model/Assignment15_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Create the Model**

**Initialization function similar to Pytorch**

In [0]:
#Initialization function similar to Pytorch
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
#Single Convolution Block
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
#Residual block
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
# Defing the David Net Model.

class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
#Cutout Function 1
from PIL import Image
class Cutout(object):
    def __init__(self, length=10):
        self.length = length

    def __call__(self, p, img):
      cutout_im = np.zeros_like(img)
      for i in range(img.shape[0]):
        p_1 = np.random.rand() 
        im = img[i]
        im = np.array(im)
        if p_1 > p:
          cutout_im[i] = im
        else:
          mask_val = im.mean()

          top = np.random.randint(0, im.shape[0])
          left = np.random.randint(0, im.shape[1])
          bottom = top + self.length
          right = left + self.length

          im[top:bottom, left:right, :] = mask_val

          im = Image.fromarray(im)
          cutout_im[i] = im
      return cutout_im

cutout=Cutout()

In [0]:
#Cutout Function 2

def random_erasing(img, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3):
    '''
    img is a 3-D variable (ex: tf.Variable(image, validate_shape=False) ) and  HWC order
    '''
    # HWC order
    
    with tf.init_scope():

      img = tf.Variable(tf.convert_to_tensor(img))
      height = tf.shape(img)[0]
      width = tf.shape(img)[1]
      channel = tf.shape(img)[2]
      area = tf.cast(width*height, tf.float32)

      erase_area_low_bound = tf.cast(tf.round(tf.sqrt(sl * area * r1)), tf.int32)
      erase_area_up_bound = tf.cast(tf.round(tf.sqrt((sh * area) / r1)), tf.int32)
      h_upper_bound = tf.minimum(erase_area_up_bound, height)
      w_upper_bound = tf.minimum(erase_area_up_bound, width)

      h = tf.random.uniform([], erase_area_low_bound, h_upper_bound, tf.int32)
      w = tf.random.uniform([], erase_area_low_bound, w_upper_bound, tf.int32)

      x1 = tf.random.uniform([], 0, height+1 - h, tf.int32)
      y1 = tf.random.uniform([], 0, width+1 - w, tf.int32)

      erase_area = tf.cast(tf.random.uniform([h, w, channel], 0, 255, tf.int32), tf.uint8)

      erasing_img = img[x1:x1+h, y1:y1+w, :].assign(erase_area)

      return tf.cond(tf.random.uniform([], 0, 1) > probability, lambda: img, lambda: erasing_img)